# Paraphrase minilm l6 vs avg glove embedding, STS comparison

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os


sys.path.append(os.path.join(os.path.abspath(""), ".."))

In [5]:
from nlp_utils.sentences.datasets import load_sts_dataset


# fetch data
sts_train, sts_dev, sts_test = load_sts_dataset()

fetching dataset...


100%|██████████| 392k/392k [00:01<00:00, 214kB/s]


In [7]:
from nlp_utils.sentences.embeddings import run_evaluator
from sentence_transformers import SentenceTransformer


model_name = "paraphrase-MiniLM-L6-v2"
sts_model = SentenceTransformer(model_name)
run_evaluator(sts_model, model_name)

dataset already exsits...
evaluating model...


,epoch,steps,cosine_pearson,cosine_spearman,euclidean_pearson,euclidean_spearman,manhattan_pearson,manhattan_spearman,dot_pearson,dot_spearman
0,-1,-1,0.836131,0.841234,0.808878,0.806445,0.806642,0.803589,0.639891,0.634872


In [8]:
glove_model_name = "average_word_embeddings_glove.6B.300d"
glove_model = SentenceTransformer(glove_model_name)

run_evaluator(glove_model, glove_model_name)

dataset already exsits...
evaluating model...


,epoch,steps,cosine_pearson,cosine_spearman,euclidean_pearson,euclidean_spearman,manhattan_pearson,manhattan_spearman,dot_pearson,dot_spearman
0,-1,-1,0.621784,0.615356,0.598526,0.614718,0.605307,0.617694,0.32855,0.321752


In [11]:
import pandas as pd
from nlp_utils.sentences.embeddings import label_sts_dataset


_, _, test_sts = load_sts_dataset(as_pd=True)

# run parahrase model over the sts dataset
para_sts = label_sts_dataset(
    test_sts.copy(),
    sts_model.encode,
    group_name="para")

# run avg glove embeddings over the sts dataset
glove_sts = label_sts_dataset(
    test_sts.copy(),
    glove_model.encode,
    group_name="glove")

# concat both dfs
sts_compare = pd.concat([para_sts, glove_sts])

sts_compare

dataset already exsits...
labelling dataset...
labelling dataset...


c:\Users\dallym\OneDrive - Genesis Energy Limited\Documents\Projects\nlp-utils\examples\..\nlp_utils\sentences\embeddings.py:20: RuntimeWarning: invalid value encountered in float_scalars
  return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))


,sentence_1,sentence_2,score,embedding_1,embedding_2,distance,residual,abs_residual,sq_residual,group
0,A girl is styling her hair.,A girl is brushing her hair.,0.50,"[-0.38507223, -0.113133855, 0.8537289, 0.15598...","[-0.6556563, -0.5492655, 0.8659563, 0.32990122...",0.841474,-0.341474,0.341474,0.116604,para
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,0.72,"[0.4808387, -0.36288857, -0.1388631, -0.482219...","[0.45801952, -0.110125065, -0.36435467, -0.682...",0.920856,-0.200856,0.200856,0.040343,para
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,1.00,"[-0.645582, -0.15931864, 0.39553022, 0.2325104...","[-0.71861655, -0.27694318, 0.5286548, 0.190856...",0.964936,0.035064,0.035064,0.001229,para
3,A man is cutting up a cucumber.,A man is slicing a cucumber.,0.84,"[-0.13937625, -0.1742496, -0.40574193, -0.0291...","[-0.43226373, -0.023343526, -0.5703407, -0.072...",0.949556,-0.109556,0.109556,0.012003,para
4,A man is playing a harp.,A man is playing a keyboard.,0.30,"[0.3796231, -0.5534441, 0.23853657, -0.4973296...","[-0.46043846, -0.6508879, -0.120257154, -0.741...",0.549526,-0.249526,0.249526,0.062263,para
...,...,...,...,...,...,...,...,...,...,...
1374,"Philippines, Canada pledge to further boost re...",Philippines saves 100 after ferry sinks,0.00,"[0.1401088, -0.019087195, -0.377794, -0.277381...","[0.045997605, -0.30591002, -0.08622762, -0.171...",0.456290,-0.456290,0.456290,0.208201,glove
1375,Israel bars Palestinians from Jerusalem's Old ...,"Two-state solution between Palestinians, Israe...",0.20,"[0.007146603, -0.25533217, -0.0453436, -0.1223...","[0.105546504, -0.068168335, 0.40202633, 0.0313...",0.670695,-0.470695,0.470695,0.221553,glove
1376,How much do you know about Secret Service?,Lawmakers from both sides express outrage at S...,0.20,"[-0.47544667, -0.004087001, 0.11432467, 0.0304...","[-0.15920566, -0.0880435, 0.13642517, -0.03754...",0.745730,-0.545730,0.545730,0.297821,glove
1377,Obama Struggles to Soothe Saudi Fears As Iran ...,Myanmar Struggles to Finalize Voter Lists for ...,0.00,"[0.23641124, 0.097179875, -0.044051237, -0.058...","[0.049643718, 0.06588553, 0.158893, -0.0402242...",0.623596,-0.623596,0.623596,0.388871,glove


In [13]:
from nlp_utils.sentences.visualisation import scatter_compare_embeddings

scatter_compare_embeddings(sts_compare)

In [14]:
scatter_compare_embeddings(sts_compare, dist_col="sq_residual")

In [18]:
import plotly.express as px


h_fig = px.histogram(
    sts_compare,
    x="sq_residual",
    facet_col="group",
    nbins=80)

h_fig.update_layout(
    margin=dict(l=16, r=16, t=16, b=16),
    height=768, width=1366
)

h_fig

In [29]:
for g in sts_compare.group.unique():
    print(g)

sts_compare

para
glove


,sentence_1,sentence_2,score,embedding_1,embedding_2,distance,residual,abs_residual,sq_residual,group
0,A girl is styling her hair.,A girl is brushing her hair.,0.50,"[-0.38507223, -0.113133855, 0.8537289, 0.15598...","[-0.6556563, -0.5492655, 0.8659563, 0.32990122...",0.841474,-0.341474,0.341474,0.116604,para
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,0.72,"[0.4808387, -0.36288857, -0.1388631, -0.482219...","[0.45801952, -0.110125065, -0.36435467, -0.682...",0.920856,-0.200856,0.200856,0.040343,para
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,1.00,"[-0.645582, -0.15931864, 0.39553022, 0.2325104...","[-0.71861655, -0.27694318, 0.5286548, 0.190856...",0.964936,0.035064,0.035064,0.001229,para
3,A man is cutting up a cucumber.,A man is slicing a cucumber.,0.84,"[-0.13937625, -0.1742496, -0.40574193, -0.0291...","[-0.43226373, -0.023343526, -0.5703407, -0.072...",0.949556,-0.109556,0.109556,0.012003,para
4,A man is playing a harp.,A man is playing a keyboard.,0.30,"[0.3796231, -0.5534441, 0.23853657, -0.4973296...","[-0.46043846, -0.6508879, -0.120257154, -0.741...",0.549526,-0.249526,0.249526,0.062263,para
...,...,...,...,...,...,...,...,...,...,...
1374,"Philippines, Canada pledge to further boost re...",Philippines saves 100 after ferry sinks,0.00,"[0.1401088, -0.019087195, -0.377794, -0.277381...","[0.045997605, -0.30591002, -0.08622762, -0.171...",0.456290,-0.456290,0.456290,0.208201,glove
1375,Israel bars Palestinians from Jerusalem's Old ...,"Two-state solution between Palestinians, Israe...",0.20,"[0.007146603, -0.25533217, -0.0453436, -0.1223...","[0.105546504, -0.068168335, 0.40202633, 0.0313...",0.670695,-0.470695,0.470695,0.221553,glove
1376,How much do you know about Secret Service?,Lawmakers from both sides express outrage at S...,0.20,"[-0.47544667, -0.004087001, 0.11432467, 0.0304...","[-0.15920566, -0.0880435, 0.13642517, -0.03754...",0.745730,-0.545730,0.545730,0.297821,glove
1377,Obama Struggles to Soothe Saudi Fears As Iran ...,Myanmar Struggles to Finalize Voter Lists for ...,0.00,"[0.23641124, 0.097179875, -0.044051237, -0.058...","[0.049643718, 0.06588553, 0.158893, -0.0402242...",0.623596,-0.623596,0.623596,0.388871,glove


In [39]:
from nlp_utils.sentences.visualisation import run_umap


def sts_compare_to_long(df):
    output_dfs = []
    for embed_group in df.group.unique():
        group_subset = df[df.group == embed_group].copy()
        e1_subset = group_subset[[
            "sentence_1",
            "embedding_1",
            "score",
            "distance",
            "group"
        ]]

        e2_subset = group_subset[[
            "sentence_2",
            "embedding_2",
            "score",
            "distance",
            "group"
        ]]

        e1_subset.rename(
            columns={
                "sentence_1": "sentence",
                "embedding_1": "embedding"
            },
            inplace=True)
        e2_subset.rename(
            columns={
                "sentence_2": "sentence",
                "embedding_2": "embedding"
            },
            inplace=True)
        output_dfs.append(pd.concat([e1_subset, e2_subset]))

    return output_dfs



sts_long = sts_compare_to_long(sts_compare)
paraphrase_df = sts_long[0]
glove_df = sts_long[1]

run_umap(paraphrase_df, n_components=3)
run_umap(glove_df, n_components=3)

C:\Users\dallym\AppData\Local\Continuum\anaconda3\envs\s_transformers\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,sentence,embedding,score,distance,group,component_1,component_2,component_3
0,A girl is styling her hair.,"[0.00039332607, -0.11147001, -0.39689335, -0.1...",0.50,0.824184,glove,2.069152,7.070474,-1.396389
1,A group of men play soccer on the beach.,"[-0.1784254, 0.46745, -0.086242005, 0.01166161...",0.72,0.940871,glove,0.931142,6.543426,0.526586
2,One woman is measuring another woman's ankle.,"[-0.30853, -0.0087560015, -0.18910666, -0.3298...",1.00,0.837407,glove,2.856359,5.831614,1.172801
3,A man is cutting up a cucumber.,"[-0.0025566618, -0.19948, 0.27462333, 0.023628...",0.84,0.890127,glove,-0.629687,3.156890,-4.003469
4,A man is playing a harp.,"[-0.25269338, 0.21950333, -0.505419, 0.0011510...",0.30,0.821381,glove,-2.042560,14.628774,7.082276
...,...,...,...,...,...,...,...,...
1374,Philippines saves 100 after ferry sinks,"[0.045997605, -0.30591002, -0.08622762, -0.171...",0.00,0.456290,glove,2.220539,4.731043,3.196534
1375,"Two-state solution between Palestinians, Israe...","[0.105546504, -0.068168335, 0.40202633, 0.0313...",0.20,0.670695,glove,2.322554,2.482468,2.829840
1376,Lawmakers from both sides express outrage at S...,"[-0.15920566, -0.0880435, 0.13642517, -0.03754...",0.20,0.745730,glove,3.633654,3.401953,2.810501
1377,Myanmar Struggles to Finalize Voter Lists for ...,"[0.049643718, 0.06588553, 0.158893, -0.0402242...",0.00,0.623596,glove,3.754925,3.111135,3.431873


In [40]:
from nlp_utils.sentences.visualisation import vis_components

vis_components(paraphrase_df, zcol="component_3", colour_col="score")

In [41]:
vis_components(glove_df, zcol="component_3", colour_col="score")